In [1]:
import  warnings
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys 
from    selenium.webdriver.common.action_chains import ActionChains
from    selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from    bs4      import BeautifulSoup
import  time
import  pyperclip
import  requests
import  datetime
import  pymssql
import  pandas as pd
import  chromedriver_autoinstaller
import  subprocess
from    selenium import webdriver
from    selenium.webdriver.chrome.options import Options
from    selenium.webdriver.common.alert import Alert
import  chromedriver_autoinstaller
import  subprocess
import  shutil 
import  xlrd
import  openpyxl 

warnings.filterwarnings('ignore')

f =  open('C:/Users/Gravity/Desktop/python/raw/itembay_ragnarok.csv', 'w', encoding='utf-8-sig')
f.write("game,server,is_trade_finished,zeny_sell_title,zeny,cash,regdate,"+'\n')

try:

    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제

except FileNotFoundError:

    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

option = Options()

option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]

try:

    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

except:

    chromedriver_autoinstaller.install(True)

    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

driver.implicitly_wait(10)

url = 'https://trade.itembay.com/login/loginAdult'

driver.get(url) #URL 가져오기
driver.implicitly_wait(10)
action = ActionChains(driver) #액션지정

#id, pw 입력할 곳 찾기

tag_id = driver.find_element_by_name('txtMemberID')
tag_pw = driver.find_element_by_name('txtPassword')
driver.implicitly_wait(10)

#id 입력

tag_id.click()
pyperclip.copy('id')
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)
driver.implicitly_wait(10)

# pw 입력

tag_pw.click()
pyperclip.copy('pw')
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)
driver.implicitly_wait(10)

#로그인 버튼 클릭

login_btn = driver.find_element_by_class_name('btn_login')
login_btn.click()
time.sleep(3)

driver.implicitly_wait(10)

ragnarok_zeny_url = 'https://trade.itembay.com/item/sell/itemSellList?iGameSeq=62&iGameServerSeq=0&tiItemType=4'

#ragnarok_zeny_url='https://trade.itembay.com/item/sell/itemSellList?iGameSeq=3026&iGameServerSeq=0&tiItemType=4'

driver.get(ragnarok_zeny_url)
time.sleep(3) #3초 있다가
driver.implicitly_wait(10)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

maximum = 0
page = 1

while 1:
    page_list = soup.find_all("a", {"id" : "btnPaging" + str(page)})
    if not page_list :
        maximum = page - 1
        break
    page = page + 1

for page in range(1, maximum+1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all("tr", {"class" : "hover_td deal_fin"})
    for ar in results :
        game_server = ar.find("a", {"class" : "server_name"}).text #게임 및 서버명
        game_server2 = game_server.replace("-", ",")
        game_server3 = game_server2.strip()
        trade_results = ar.find("td", {"class": "deal_fin_ico"}).text #거래완료 여부
        zeny_selling_title = ar.find("td", {"class": "goods_name"}).text #제니판매 글 제목
        zeny_selling_title2 = zeny_selling_title.replace(",", "")
        zeny = ar.td.next_sibling.next_sibling.next_sibling.text #판매제니수량
        zeny2 = zeny.replace(",", "")
        cash = ar.td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text #거래가격
        cash2 = cash.replace(",", "")
        cash3 = cash2.replace("원", "")
        cash4 = cash3.strip()
        date = ar.td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text #등록일
        f.write(game_server3+','+ trade_results +','+ zeny_selling_title2 +','+ zeny2  +','+  cash4 +','+ date + '\n')  
    try :
        page = page +1
        driver.find_element_by_xpath('//*[@id="btnPaging'+str(page)+'"]').click()
        time.sleep(2)
        driver.implicitly_wait(10)
    except :
        print("end")
        break

f.close()

driver.close()   
driver.quit()

end
